In [1]:
import math
import logging
FORMAT = '[%(name)s:%(levelname)s]  %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT)
logger = logging.getLogger('dbg')

def dprint(s):
    logger.debug(s)

def iprint(s):
    logger.info(s)

logger.setLevel(logging.INFO)

## Red Black BST Trees

**Red Black Trees** (RBTs) are a modified BST, that add rules to ensure insertions and deletions result in more balanced final tree structures, hence they are known as 'self-balancing'. As many of the tasks of BSTs are $O(h)$, this gives assuredly fast _search, insertion and deletion_ operations.

In a RST each node has a color (Red or Black) and if a node has no left or right child, these point to a common "Nil" Node, that is logically black. The tree is always approximately balanced with Tree Height $h = \Theta( \log n)$.

<img src="media/RBT.png" alt="drawing" width="450"/>

The result is as follows (with the addition storage remains $\Theta(n)$):

| Complexity | Worst Case |
| ---------- | ---------- |
| Search     | $O(\log n)$ |
| Insert     | $O(\log n)$ |
| Delete     | $O(\log n)$ |


### RBT Rules:

The rules or properties that must be satisfied are as follows:

| Number    | Property |
| --------- | -------- |
| 1         | Every node is *red* or *black* |
| 2         | The root node is *black* |
| 3         | The 'Nil' node is *black* |
| 4         | If a node is *red* then both children are *black* |
| 5         | From any node, all paths to leaf nodes have equal numbers of *black* nodes |

Therefore, all root-leaf paths have the same number of *black* nodes, and all *red* nodes have black children.
This places a limit on the maximum size of the longest path (height):

$ h_{max} \leq 2 \cdot h_{min} $

Noting that a perfectly balanced tree has $ n = 2^{h+1} -1 $ nodes implies $ h_{min} \leq \log_2 (n + 1) \therefore h_{min} = \Theta( \log_2 n)$.

$ h_{min} \leq h_{max} \leq 2 \cdot h_{min} \Rightarrow h_{max} = \Theta( \log_2 n)$ == self balancing result!

